In [1]:
import pandas as pd
from difflib import get_close_matches
from difflib import SequenceMatcher

In [2]:
df1 = pd.read_excel("../Data/DDW_PCA0000_2011_Indiastatedist.xlsx", sheet_name='Sheet1', usecols = ['State', 'Level', 'Name', 'TRU', 'TOT_P', 'TOT_M', 'TOT_F'])
df1 = df1[df1["TRU"] == "Total"]
#Replacing State codes with state names in census data
for idx, item in df1[df1["Level"] == "STATE"].iterrows():
    df1.replace(item["State"], item["Name"], inplace = True)
df1 = df1[df1["Level"] == "DISTRICT"]
df1.reset_index(inplace = True)

In [3]:
df2 = pd.read_csv("../Data/cowin_vaccine_data_districtwise.csv", skiprows = [1])
df2 = df2.fillna(0)
df2 = df2.groupby(['State', 'District_Key', 'District'], as_index=False).agg('sum')

In [4]:
#Replacing names in census data with district id
patterns = list(df2["District"])
count = 0
for idx, word in enumerate(df1["Name"]):
    match = get_close_matches(word, patterns, cutoff = 0.75)
    maxr = 0
    name = ''
    for item in match:
        r = SequenceMatcher(None, item, word).ratio()
        if(r > maxr):
            name = df2.District_Key[df2["District"] == item].astype('category').values[0]
            maxr = r
        elif r == maxr:
            state = df1.State[df1["Name"] == word].astype('category').values[0]            #State of the district in CENSUS Data
            state1 = df2.State[df2["District"] == item].astype('category').values[0]       #State of the district in Vaccine Data
            
            if state == state1.upper():
                name = df2.District_Key[df2["District"] == item].astype('category').values[0]
                maxr = r
        #print(word)
    if name == '':
        df1.drop(index = idx, inplace = True)
    else :
        df1['Name'][idx] = name
df1.reset_index(inplace = True)

C:\Users\Dinkar\AppData\Local\Temp/ipykernel_13236/3781070227.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Name'][idx] = name


In [9]:
dist = {"districtid" : [], "vaccinationratio" : [], "populationratio" : [], "ratioofratios" : []}
for idx, row in df1.iterrows():
    dist["districtid"].append(row["Name"])
    
    #vaccination ratio calculation
    f = df2[df2["District_Key"] == row["Name"]]["14-08-2021.6"].astype('category').values[0]
    m = df2[df2["District_Key"] == row["Name"]]["14-08-2021.5"].astype('category').values[0]
    
    dist["vaccinationratio"].append(f/m)
    
    #population ratio calculation
    pratio = row["TOT_F"] / row["TOT_M"]
    dist["populationratio"].append(pratio)
    
    #ratio of ratios
    rr = dist["vaccinationratio"][idx] / dist["populationratio"][idx]
    dist["ratioofratios"].append(rr)

In [10]:
df = pd.DataFrame(dist)

In [11]:
df.to_csv("../Final Outputs/vaccination-population-ratio-dist.csv", index = False)